In [1]:
# import packages
import pandas as pd
import numpy as np 
import re 
import time
import json

# Reddit

In [2]:
import praw
import requests
import pickle

pickle_in = open("individual_info","rb")
individual_info = pickle.load(pickle_in)

In [3]:
reddit = praw.Reddit(username=individual_info['username'],
                      password=individual_info["password"],
                     client_id=individual_info['client_id'], 
                     client_secret=individual_info['client_secret'], 
                     user_agent=individual_info['user_agent'],
                    )

In [4]:
def extract_data(topic):
    subreddit=reddit.subreddit(topic)
    hot_python = subreddit.hot(limit=1000)
    title_reply=[]
    for submission in hot_python:
        if not submission.stickied:
            comments = submission.comments
            for comment in comments:
                try:
                    title_reply.append({"body":comment.body,
                                        "reply":[reply.body for reply in comment.replies]})                 
                except:
                    pass
                
                
                
    questions=[]
    answers=[]
    for i in range(len(title_reply)):
        questions.append(list(title_reply)[i]['body'])
        answers.append(list(title_reply)[i]["reply"]) 
        
    return questions,answers

In [5]:
# extract data for those topics in the topics1 list 
topics1=["datascience","Python","pythoncoding",'python_netsec',"pythontips","pythonprojects2"]
Questions=[]
Answers=[]
for topic in topics1:
    q,a=extract_data(topic)
    Questions.append(q)
    Answers.append(a)

In [6]:
topics2=['Numpy','statsmodels',"matplotlib","datacleansing","anaconda","datavisualization"]
for topic in topics2:
    q,a=extract_data(topic)
    Questions.append(q)
    Answers.append(a)

In [43]:
len(Questions),len(Answers)

(12, 12)

In [44]:
questions=[]
answers0=[]
for i in range(len(Questions)):
    for j in range(len(Questions[i])):
        questions.append(Questions[i][j])
        answers0.append(Answers[i][j])

answers=[]
for i in answers0:
    try:
        answers.append(i[0])
    except:
        answers.append(None)

In [45]:
len(questions),len(answers)

(10236, 10236)

In [46]:
questions[1],answers[1]

('This is so interesting my god I love it',
 'Yeah, I love OPs style so much and have watched all of his videos! Recommend them to everyone I know as well, are so well made!')

In [36]:
pickle_out1=open('questions', 'wb')
pickle_out2=open('answers', 'wb')

pickle.dump(reddit, pickle_out1)
pickle.dump(reddit, pickle_out2)

pickle_out1.close()
pickle_out2.close()

In [37]:
questions = pickle.load(open("questions","rb"))
answers = pickle.load(open("answers","rb"))

In [47]:
df=pd.DataFrame()
df['question']=questions
df["answer"]=answers

In [48]:
df.head()

,question,answer
0,I've also used plotly with the dash module in ...,"Plus you can build Dash apps with R now too, s..."
1,This is so interesting my god I love it,"Yeah, I love OPs style so much and have watche..."
2,I used plotly for an R Shiny app recently and ...,None
3,I watched this video last night at 2 AM and it...,Thanks so much! That's been my main goal with ...
4,Also check highcharter and r2d3 for even bette...,I second r2d3. I've always found plotly less s...


In [49]:
df.to_csv("df_python_reddit.csv")